In [1]:
from datetime import date
import requests
import pandas as pd

In [2]:
with open('.env') as file:
    env = {key: value for key, value in (line.split('=') for line in file.read().split('\n') if line)}

In [16]:
headers = {
    'Authorization': f'apikey {env["twelve-key"]}'
}

In [17]:
base_url = 'https://api.twelvedata.com'

In [18]:
r = requests.get(base_url + '/earliest_timestamp', {
    'symbol': 'AAPL',
    'interval': '1min'
}, headers=headers)
assert r.status_code == 200

In [19]:
r.json()

{'datetime': '2020-03-24 10:07:00', 'unix_time': 1585058820}

In [20]:
r = requests.get(base_url + '/earliest_timestamp', {
    'symbol': 'A',
    'interval': '1min'
}, headers=headers)
assert r.status_code == 200
r.json()

{'datetime': '2020-03-23 14:47:00', 'unix_time': 1584989220}

In [46]:
r = requests.get(base_url + '/time_series', {
    'symbol': ','.join(['AAPL']),
#     'interval': '1min'
    'interval': '1day',
    'start_date': str(date(2023, 1, 10)),
    'end_date': str(date(2023, 1, 10))
}, headers=headers)
assert r.status_code == 200

In [43]:
def add_ticker(ticker, to):
    to['ticker'] = ticker
    return to

In [47]:
r.json()

{'code': 400,
 'message': 'No data is available on the specified dates. Try setting different start/end dates.',
 'status': 'error',
 'meta': {'symbol': 'AAPL', 'interval': '1day', 'exchange': ''}}

In [45]:
add_ticker(r.json()['meta']['symbol'], pd.DataFrame(r.json()['values']))

,datetime,open,high,low,close,volume,ticker
0,2023-01-10,130.25999,131.26360,128.12000,130.73000,63428000,AAPL


In [41]:
pd.DataFrame([
    j for i in
        ((add_ticker(key, row) for row in data['values'])
         for key, data in r.json().items())
    for j in i
])

KeyError: 'values'

---

In [3]:
from portfolio_rl.data_sources.twelve_client import TwelveClient

In [5]:
client = TwelveClient(env["twelve-key"], 'data/twelve_cache')

In [9]:
client.cached_single_time_series('AAPL', date(2023, 1, 10), date(2023, 1, 12), interval='1day')

Found cached file data/twelve_cache/twelve_AAPL_20230110_20230112_1day.parquet


,datetime,open,high,low,close,volume,ticker,date
0,2023-01-10,130.25999,131.26360,128.12000,130.73000,63428000.0,AAPL,2023-01-10
1,2023-01-11,131.25000,133.50999,130.46001,133.49001,69120000.0,AAPL,2023-01-11
